In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

matplotlib.rc("axes", unicode_minus = False)

In [3]:
df1 = pd.read_csv("01 Oxidation.csv")
df2 = pd.read_csv("02 Photo_softbake.csv")
df3 = pd.read_csv("03 Photo_lithograpy.csv")
df4 = pd.read_csv("04 Etching.csv")
df5 = pd.read_csv("05 Ion_Implantation.csv")
df6 = pd.read_csv("06 Inspect.csv")

In [4]:
df = pd.merge(df1, df2, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df3, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df4, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df5, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df6, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])

In [5]:
df.dropna(subset="Thin F2", inplace=True) # 다른 변수들과의 연관성을 찾지못함, 한 행에 여러 열들 결측값 가짐
# pd.set_option('display.max_row', 200)
# pd.set_option('display.max_columns', 200)

In [6]:
df['Ox_Chamber'] = df['Ox_Chamber'].astype('str')
df['photo_soft_Chamber'] = df['photo_soft_Chamber'].astype('str')
df['lithography_Chamber'] = df['lithography_Chamber'].astype('str')
df['Etching_Chamber'] = df['Etching_Chamber'].astype('str')
df['Chamber_Num'] = df['Chamber_Num'].astype('str')
df['path'] = df['Ox_Chamber']+df['photo_soft_Chamber']+df['lithography_Chamber']+df['Etching_Chamber']+df['Chamber_Num']

In [7]:
df.drop(df[df['Oxid_time']<0].index, inplace=True) # 산화시간이 음수
df.drop(df[df['Target']==0].index, inplace=True) # target 값이 0
df.drop(columns = ["Wafer_map","Error_message"],inplace=True) # 웨이퍼맵, 에러메시지

In [8]:
# 시간 데이터 datetime 유형으로 변환
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y')

# 195이상 = 불량(1), 195미만 = 양품(0)
df.loc[df['Target'] >= 195, '불량_195이상'] =1
df.loc[df['Target'] < 195, '불량_195이상'] =0

In [9]:
df.loc[df['Thin F4']<0, 'Thin F4']=df['Thin F4'].median()
df.loc[df['Flux90s']<0, 'Flux90s']=df['Flux90s'].median()
df.loc[df['Flux160s']<0, 'Flux160s']=df['Flux160s'].median()
df.loc[df['Flux160s']<5, 'Flux160s']=df['Flux160s'].median()

In [10]:
df['Thin F1'].fillna(df['Thin F1'].median(), inplace=True)
df['Thin F3'].fillna(df['Thin F3'].median(), inplace=True)
df['Flux60s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux90s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux480s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux840s'].fillna(df['Flux90s'].median(), inplace=True)

### 불필요한 열 제거

In [11]:
# 불필요한 열 제거 
# df.drop(columns=["No_Die", "Lot_Num", "Wafer_Num"], inplace=True)
df.drop(columns=['Vapor','process','Wavelength'])
df.loc[df['Flux840s'] == 8.137500e+16, 'Flux840s'] = df['Flux840s'].mode()[0]
df.loc[df['Flux480s'] == 8.137500e+16, 'Flux480s'] = df['Flux480s'].mode()[0]

In [12]:
df_new = df[['Temp_OXid','ppm','Pressure','type','Oxid_time',
        'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'UV_type', 'Energy_Exposure',       
        'Temp_Etching', 'Source_Power',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       '불량_195이상', 'path']]

In [13]:
# 데이터 구성:Series, DataFrame
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib

# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV

# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [14]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

df_raw_x = df_new.drop("불량_195이상", axis = 1)
df_raw_y = df_new["불량_195이상"] 

# object 형식의 열들에 대해 Label Encoding 수행
for column in df_raw_x.columns:
    if df_raw_x[column].dtype == 'object':
        df_raw_x[column] = label_encoder.fit_transform(df_raw_x[column])

from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# 데이터 스케일링
X_scaled = scaler.fit_transform(df_raw_x)

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
    X_scaled, df_raw_y, test_size = 0.3, random_state = 1234, stratify=df_raw_y) 

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 전처리
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_raw_x)  # 특성 스케일링

X_train, X_test, y_train, y_test = train_test_split(X_scaled, df_raw_y,
                                                    test_size=0.3, random_state=1234, stratify=df_raw_y )  # 훈련/검증 데이터 분할

# 모델 구성
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # 입력 레이어
    Dropout(0.5),  # 드롭아웃 레이어
    Dense(64, activation='relu'),  # 은닉 레이어
    Dropout(0.5),  # 드롭아웃 레이어
    Dense(32, activation='relu'),  # 추가된 은닉 레이어
    BatchNormalization(),  # 배치 정규화 레이어 추가
    Dense(1, activation='sigmoid')  # 출력 레이어 (이진 분류이므로 시그모이드 활성화 함수 사용)
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # 이진 분류이므로 binary crossentropy 사용
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')



Epoch 1/100


30/30 [==============================] - 1s 8ms/step - loss: 0.7979 - accuracy: 0.5359 - val_loss: 0.5884 - val_accuracy: 0.8987
Epoch 2/100
30/30 [==============================] - 0s 4ms/step - loss: 0.6301 - accuracy: 0.6572 - val_loss: 0.5006 - val_accuracy: 0.9241
Epoch 3/100
30/30 [==============================] - 0s 4ms/step - loss: 0.5560 - accuracy: 0.7511 - val_loss: 0.4366 - val_accuracy: 0.9367
Epoch 4/100
30/30 [==============================] - 0s 3ms/step - loss: 0.4857 - accuracy: 0.8228 - val_loss: 0.3719 - val_accuracy: 0.9367
Epoch 5/100
30/30 [==============================] - 0s 3ms/step - loss: 0.4342 - accuracy: 0.8597 - val_loss: 0.3233 - val_accuracy: 0.9367
Epoch 6/100
30/30 [==============================] - 0s 3ms/step - loss: 0.3932 - accuracy: 0.8945 - val_loss: 0.2974 - val_accuracy: 0.9367
Epoch 7/100
30/30 [==============================] - 0s 3ms/step - loss: 0.3525 - accuracy: 0.9061 - val_loss: 0.2770 - val_accuracy: 0.9367
Epoch 8/1

In [16]:
asdasd

NameError: name 'asdasd' is not defined

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)

# 혼동 행렬 계산
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_class))

# 분류 보고서 출력
print("Classification Report:")
print(classification_report(y_test, y_pred_class))

16/16 [==============================] - 0s 1ms/step
Confusion Matrix:
[[469   3]
 [ 26  10]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       472
         1.0       0.77      0.28      0.41        36

    accuracy                           0.94       508
   macro avg       0.86      0.64      0.69       508
weighted avg       0.93      0.94      0.93       508



In [18]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [19]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # 입력 레이어
    Dropout(0.5),  # 드롭아웃 레이어
    Dense(64, activation='relu'),  # 은닉 레이어
    Dropout(0.5),  # 드롭아웃 레이어
    Dense(32, activation='relu'),  # 추가된 은닉 레이어
    BatchNormalization(),  # 배치 정규화 레이어 추가
    Dense(1, activation='sigmoid')  # 출력 레이어 (이진 분류이므로 시그모이드 활성화 함수 사용)
])


# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # 이진 분류이므로 binary crossentropy 사용
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_resampled, y_resampled, epochs=300, batch_size=32, validation_split=0.2, verbose=1)

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/300
55/55 [==============================] - 1s 6ms/step - loss: 0.7458 - accuracy: 0.5790 - val_loss: 0.6508 - val_accuracy: 0.5955
Epoch 2/300
55/55 [==============================] - 0s 3ms/step - loss: 0.6238 - accuracy: 0.6687 - val_loss: 0.6463 - val_accuracy: 0.5750
Epoch 3/300
55/55 [==============================] - 0s 3ms/step - loss: 0.5584 - accuracy: 0.7108 - val_loss: 0.6161 - val_accuracy: 0.5955
Epoch 4/300
55/55 [==============================] - 0s 3ms/step - loss: 0.5276 - accuracy: 0.7256 - val_loss: 0.6078 - val_accuracy: 0.6045
Epoch 5/300
55/55 [==============================] - 0s 3ms/step - loss: 0.5031 - accuracy: 0.7517 - val_loss: 0.5812 - val_accuracy: 0.6205
Epoch 6/300
55/55 [==============================] - 0s 3ms/step - loss: 0.4492 - accuracy: 0.8000 - val_loss: 0.5498 - val_accuracy: 0.6318
Epoch 7/300
55/55 [==============================] - 0s 3ms/step - loss: 0.4458 - accuracy: 0.7915 - val_loss: 0.4978 - val_accuracy: 0.6841
Epoch 8/300
5

In [20]:
from sklearn.metrics import confusion_matrix, classification_report

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)

# 혼동 행렬 계산
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_class))

# 분류 보고서 출력
print("Classification Report:")
print(classification_report(y_test, y_pred_class))

16/16 [==============================] - 0s 2ms/step
Confusion Matrix:
[[461  11]
 [ 26  10]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96       472
         1.0       0.48      0.28      0.35        36

    accuracy                           0.93       508
   macro avg       0.71      0.63      0.66       508
weighted avg       0.91      0.93      0.92       508

